### Installing Libraries

In [ ]:
!pip install -q cassio datasets langchain openai tiktoken

In [ ]:
pip install faiss-cpu


In [ ]:
import faiss


### Import the packages

In [ ]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from datasets import load_dataset
import cassio

In [ ]:
!pip install PyPDF2

In [ ]:
from PyPDF2 import PdfReader

### Setup all the tokens and api keys


In [ ]:
import os

In [ ]:
ASTRA_DB_APPLICATION_TOKEN = "AstraCS:YiBvylgvMOanmFjYxjTqBlAm:2265f41ae0c862fc8f8560b88e37890389c461b3b96b3bf9aa778403c0ed44b4"
ASTRA_DB_ID = "eb9700c2-fc53-43ca-988f-ad6d4df16fe9" # enter your Database ID
OPENAI_API_KEY ="sk-proj-6MJfPIj5KDBPBxzCuxBBT3BlbkFJmL198mgSzQLxuVh1yB3y" # enter your OpenAI key

### provide the path of  pdf file/files.


In [ ]:
pdfreader = PdfReader('/content/sodapdf-converted.pdf')

In [ ]:
from typing_extensions import Concatenate
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [ ]:
raw_text

'CITY==  Hyderabad  \n  \nURL: https://en.wikipedia.org/wiki/List_of_tourist_attractions_in_Hyderabad  \n  \n  \nCategory: Historical monuments  \n- Charminar ± a major landmark of Hyderabad with four graceful minarets located in the old city. It was \nbuilt by Muhammed Quli Qutb Shah as a memorial for plague victims. Charminar, on most occasions, is \nused to represent the city and the state and is hailed as a unique Deccan monument. It is in the midst of \nCharkaman which are four archways to roads leading in all four directions from the Charminar \nmonument.[9]  \n- Mecca Masjid ±  A symbolic mosque, Mecca Masjid was built 400 years ago during the reign of Sultan \nMuhammad Qutub Shah, the 6th Qutub Shahi Sultan of Hyderabad. The three-arched facade has been \ncarved from a single piece of granite, which took five years to quarry. More than 8,000 workers were \nemployed to build the mosque. Muhammed Qutub Shah personally laid the foundation stone.  \n  \nCategory: Palaces  \n- Falak

In [ ]:
len(raw_text)

518144

### Initialize the connection to your database


In [ ]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

ERROR:cassandra.connection:Closing connection <LibevConnection(134264431589936) eb9700c2-fc53-43ca-988f-ad6d4df16fe9-us-east1.db.astra.datastax.com:29042:d9190b26-3178-4820-99e1-70edbf2e75fc> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


### Create the LangChain embedding and LLM objects

In [ ]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


### Create your LangChain vector store

In [ ]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None,
)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
texts[0]

'CITY==  Hyderabad  \n  \nURL: https://en.wikipedia.org/wiki/List_of_tourist_attractions_in_Hyderabad  \n  \n  \nCategory: Historical monuments  \n- Charminar ± a major landmark of Hyderabad with four graceful minarets located in the old city. It was \nbuilt by Muhammed Quli Qutb Shah as a memorial for plague victims. Charminar, on most occasions, is \nused to represent the city and the state and is hailed as a unique Deccan monument. It is in the midst of \nCharkaman which are four archways to roads leading in all four directions from the Charminar \nmonument.[9]  \n- Mecca Masjid ±  A symbolic mosque, Mecca Masjid was built 400 years ago during the reign of Sultan \nMuhammad Qutub Shah, the 6th Qutub Shahi Sultan of Hyderabad. The three-arched facade has been'

In [ ]:
texts[1]

"- Mecca Masjid ±  A symbolic mosque, Mecca Masjid was built 400 years ago during the reign of Sultan \nMuhammad Qutub Shah, the 6th Qutub Shahi Sultan of Hyderabad. The three-arched facade has been \ncarved from a single piece of granite, which took five years to quarry. More than 8,000 workers were \nemployed to build the mosque. Muhammed Qutub Shah personally laid the foundation stone.  \n  \nCategory: Palaces  \n- Falaknuma Palace ± built by an Italian architect through one of the Paigah nobles, Nawab Viqar \nal-Umra', complete with Italian marble, Louis XIV-style decor, a Mughal ambience, Italian marble \nstaircases and ornate fountains. It has now been undertaken by Taj group to develop this as a heritage"

In [ ]:
texts[2]

"staircases and ornate fountains. It has now been undertaken by Taj group to develop this as a heritage \nhotel. It had taken 9 years to build the palace, from 1884 to 1893. The library in the Palace is said to be \nas majestic as the one in the Windsor Castle. One of the main attractions of the palace is the 100 seater \ntable that is 108 feet long  \n- Chowmahalla Palace ± It was the seat of Asaf Jahi dynasty, where the Nizam entertained his official \nguests and royal visitors. Initiated in 1750 by Nizam Salabat Jung and designed along the lines of the \nShah's palace in Isfahan, this actually consists of a group of palaces each used as a Durbar Hall. It has \nnow been aesthetically renovated and is also a venue for conferences.[12]"

In [ ]:
len(texts)

848

### Load the dataset into the vector store



In [ ]:
astra_vector_store.add_texts(texts)
print("Inserted %i headlines." % len(texts))
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 848 headlines.


### ------------------------------------------- Travel ChatBot ---------------------------------------



In [ ]:
import pandas as pd
# Flag to check if it's the first question
first_question = True

In [ ]:
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)

    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)
    print("FIRST 4 DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
      print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:1000]))
# a day trip to Hyderabad to visit Temples,lakes etc


Enter your question (or type 'quit' to exit): Give some places related to nature, museums and beaches in chennai

QUESTION: "Give some places related to nature, museums and beaches in chennai"


ANSWER: "Some places in Chennai related to nature are Aringar Anna Zoological Park in Vandalur, Valluvar Kottam, and Chennai viewed from St. Thomas Mount. Museums in Chennai include the Government Museum at Egmore and Chettinad Palace. Beaches in Chennai include Marina Beach, Elliot's Beach, and Blue Flag Beach."

FIRST 4 DOCUMENTS BY RELEVANCE:


    [0.9307] "-   
  
War Memorial  
  
-   
  
Schmidt memorial at Elliot's beach  
  
-   
  
Government Museum at Egmore  
  
-     
Valluvar Kottam  
  
-   
  
Aringar Anna Zoolological Park in Vandalur  
  
-   
  
The Victoria Public Hall  
  
  
Category: Gallery  
-   
  
Santhome Basilica  
  
-   
  
Interior of Santhome Basilica  
  
-   
  
Chettinad Palace  
  
-   
  
Chennai viewed from St. Thomas Mount  
  
-   
  
War Memorial  
  
-   
  
Schmidt memorial at Elliot's beach  
  
-   
  
Government Museum at Egmore  
  
-   
  
Valluvar Kottam  
  
-   
  
Aringar Anna Zoolological Park in Vandalur  
  
-   
  
The Victoria Public Hall  
  
  
Category: See also  - India portal  
  
Category: References  
- Places to visit in Chennai  
- Oneday Trip Around Chennai ..."
    [0.9307] "-   
  
War Memorial  
  
-   
  
Schmidt memorial at Elliot's beach  
  
-   
  
Government Museum at Egmore  
  
-     
Valluvar Kottam  
  
-   
  
Aringar Anna Zoolological Park in Vand